In [5]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

In [6]:
# Load API key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:
system_message = """
You are a robotics task planner operating in a standard household environment.
Your job is to break down each high-level command from the user into a clear,
step-by-step plan that a mobile robot manipulator could realistically execute and provide only and only instructions.

Assume the robot can:
1. Navigate indoor spaces (e.g., living room, kitchen).
2. Detect and identify common household objects (cups, cans, snacks).
3. Pick up and place items with a gripper.
4. Use basic cleaning tools (e.g., a cloth or paper towel) if needed.

Each step should reflect physically feasible actions in logical order.
If a user’s request is ambiguous, clarify or make reasonable assumptions
and note them. Return the plan as a concise, numbered list with clear instructions.
Avoid extraneous commentary.
"""

In [14]:
prompts = [
  "I spilled my drink, Can you help?",
  "How would you bring me a coke can?",
  "Can you bring me a snack from the table?",
  "I'm feeling cold; can you bring me a blanket from the closet in the hallway?"
]

In [15]:
# Function to query GPT-4o
def generate_steps(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

In [16]:
# Run experiments and save results
results = []
for idx, prompt in enumerate(prompts):
    response = generate_steps(prompt)
    results.append({"id": idx+1, "prompt": prompt, "response": response})
    print(f"Prompt {idx+1}:\n{response}\n{'-'*50}")

# Save results to JSON
with open("llm_task_decomposition_results.json", "w") as f:
    json.dump(results, f, indent=2)

Prompt 1:
1. Navigate to the location of the spill.
2. Identify the spilled liquid and the surrounding area.
3. Locate a cleaning cloth or paper towel in the vicinity (e.g., kitchen or cleaning supplies area).
4. Navigate to the cleaning cloth or paper towel and pick it up with the gripper.
5. Return to the spill location.
6. Place the cleaning cloth or paper towel on the spill to absorb the liquid.
7. Press down gently on the cloth or paper towel to ensure maximum absorption.
8. If necessary, repeat steps 6 and 7 with a new cloth or paper towel until the spill is fully absorbed.
9. Dispose of the used cleaning cloth or paper towel in the trash.
10. Return to the charging station or standby location.
--------------------------------------------------
Prompt 2:
1. Navigate to the kitchen area.
2. Use sensors to scan and identify the location of a Coke can.
3. Move to the identified location of the Coke can.
4. Position the gripper above the Coke can.
5. Lower the gripper to grasp the Co